In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
import sklearn
from collections import Counter
from collections import defaultdict
from tqdm import tqdm
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
import pandas as pd

# 各種import，原本想一起管理但後來懶了

In [2]:
# 字寬
from wcwidth import wcswidth

def pad_display(text, width, align='left'):
    display_width = wcswidth(text)
    pad_len = max(0, width - display_width)

    if align == 'left':
        return text + ' ' * pad_len
    elif align == 'right':
        return ' ' * pad_len + text
    elif align == 'center':
        left = pad_len // 2
        right = pad_len - left
        return ' ' * left + text + ' ' * right
    else:
        return text

# 幫助排版，免得全形字干擾字寬控制

In [3]:
# 載入難度資料
df = pd.read_csv("real_difficulty.csv") # QQ群載的
df = df.dropna(subset=['file_name'])

# 去掉無資料的部分

In [4]:
# 載入譜面資料
for i in df.index:
  role_id = df.loc[i, "role_id"]

# 全局變量，記住所有譜面資訊，有index就可以一起拿到
available_chart = []
real_diff_list = []
all_song_name = []
all_chart_json = []

# CT2View會丟失頁面參數訊息，一般來說都直接忽略，但也可能會造成譜面難以使用
# 但這裡只能放因為處理不當而壞掉的，不能單純放很難處理的
dirty_data = ["ilka001_014+chaos", "neko002_001+glitch"]
dirty_data = []

for i in df.index:
    filename = df.loc[i, "file_name"]+".json"
    if filename in dirty_data:
        continue
    if not os.path.exists("Cytus2Chart/" + filename):
        continue
    with open("Cytus2Chart/" + filename, newline='') as f: # CT2View爬的
        chart_json = json.load(f)
    all_chart_json.append(chart_json)
    available_chart.append(filename)
    real_diff_list.append(df.loc[i, "diff"])
    all_song_name.append(df.loc[i, "song_name"])
    if "vanessa001_017+chaos.json" == filename: # 測試用，不重要
        print(df.loc[i, "song_name"])


print(len(real_diff_list), len(available_chart), len(all_song_name))

Installation (VILA)
643 643 643


In [5]:
# 用tick算時間(秒)
def tick_to_seconds_precise(tick, tempo_changes, ticks_per_beat=480):
    tempo_changes = sorted(tempo_changes, key=lambda x: x['tick'])

    current_time_microsec = 0
    for i in range(len(tempo_changes) - 1):
        start_tick = tempo_changes[i]['tick']
        end_tick = tempo_changes[i+1]['tick']
        value = tempo_changes[i]['value']  # μs/拍

        if tick < start_tick:
            # 輸入 tick 在目前區間之前
            break

        if tick < end_tick:
            # tick 落在此區間內
            delta_ticks = tick - start_tick
            beats = delta_ticks / ticks_per_beat
            current_time_microsec += beats * value
            return current_time_microsec / 1_000_000

        # tick 超過此區間，累積整段區間時間
        delta_ticks = end_tick - start_tick
        beats = delta_ticks / ticks_per_beat
        current_time_microsec += beats * value

    # 如果 tick 超過最後 tempo 節點，使用最後一段速度累積時間
    last_tick = tempo_changes[-1]['tick']
    last_value = tempo_changes[-1]['value']
    delta_ticks = tick - last_tick
    beats = delta_ticks / ticks_per_beat
    current_time_microsec += beats * last_value

    return current_time_microsec / 1_000_000


In [6]:
# 算每頁的分數，不同拍點有不同的加權計分方式
def get_page_score(note_list, page_list):
    ### PAGE_SCORE
    page_scores = defaultdict(float)
    for note in note_list:
        page_id = note['page_index']
        type_ = note['type']
        hold_tick = note.get('hold_tick', 0)
        tick = note.get('tick', 0)

        if type_ == 0: # click
            page_scores[page_id] += 1
        elif type_ == 1: # hold
            page_scores[page_id] += 1 + (hold_tick / 240) * 0.1
        elif type_ == 2: # Lhold
            # 固定頭 1.5 分在起始頁
            page_scores[page_id] += 1.5
            start_tick = note['tick']
            end_tick = start_tick + note['hold_tick']
            while page_id < len(page_list):
                page = page_list[page_id]
                page_start = page['start_tick']
                page_end = page['end_tick']
                # 若頁面起點 >= LHold 結束，代表不再有交集，結束
                if page_start >= end_tick:
                    break
                # 計算與 LHold 的交集區間
                overlap_start = max(start_tick, page_start)
                overlap_end = min(end_tick, page_end)
                overlap_ticks = overlap_end - overlap_start
                # LHold時間計算分數
                page_scores[page_id] += (overlap_ticks / 240) * 0.1
                page_id += 1
        elif type_ == 3: # DragH
            page_scores[page_id] += 0.5
        elif type_ == 4: # DragC
            page_scores[page_id] += 0.2
        elif type_ == 5: # Flick
            page_scores[page_id] += 1.5
        elif type_ == 6: # CDragH
            page_scores[page_id] += 1
        elif type_ == 7: # CDragC
            page_scores[page_id] += 0.2
    return page_scores

# 測試用，但很應該不太有bug
with open("Cytus2Chart/" + "xenon001_007+glitch.json", newline='') as f:
    chart_json = json.load(f)
    note_list = chart_json["note_list"]
    note_list = sorted(note_list, key=lambda x: x['tick'])
    page_list = chart_json["page_list"]
    page_list = sorted(page_list, key=lambda x: x['start_tick'])
    get_page_score(note_list, page_list)


In [ ]:
### 分配手順，計算每顆note承受的爆發度，考慮flick需要時間處理且佔一些位移量、鎖鏈，考慮hold鎖手，偵測三壓但不考慮三壓難度
import matplotlib.pyplot as plt
import time
from collections import namedtuple


### 給兩位置的時間座標，計算出力分數，t是秒
def compute_burst(x1, x2, t1, t2, hand, a=0.2, is_debug=False, is_post_process=False, type1=0):
    dt = max(min(t2 - t1, 1), 0.001) # 時間太長太短都忽略

    # 因為原本的手順測試運作兩好，後續只想更改burst最終計算方式，所以用is_post_process避免動到手順計算
    if is_post_process and type1==5:
        dt = max(dt-0.05, 0.001) # flick處理時間長
        if hand == 'L': # flick打完後位置偏移
            x2 -= 0.1
        elif hand == 'R':
            x2 += 0.1

    dx = abs(x2 - x1)
    if is_post_process and type1==5:  # flick本身要位移
        dx += 0.1

    # 經驗公式
    time_burst = 1 / (dt ** 1.5)
    space_burst = (dx * 10) ** 1.5 / 10

    # 計算手順時讓左手在左、右手在右，但實際上算burst時可以不用管
    if not is_post_process:
        if hand == 'L':
            hand_coe = 1 + (x2 * 10)
        elif hand == 'R':
            hand_coe = 1 + ((1 - x2) * 10)
        else:
            hand_coe = 0
    else:
        hand_coe = 0

    if is_debug:
        output = [round(val, 2) for val in [space_burst, time_burst, (a + space_burst) * time_burst, hand_coe]]
        # print(output)

    v = (a + space_burst) * time_burst + hand_coe
    if is_post_process:
        v = v**0.7 # 讓太大的值小一點
        if type1 == 5: # 再幫flick多加點分數
            v = v + 0.5
    return (a + space_burst) * time_burst + hand_coe

def minimize_burst_with_path(points, a=0.2):
    from collections import defaultdict
    import math
    is_three_hand = False

    # 神奇的動態規劃，我也不是太懂，大都ChatGPT寫的
    MAX_STATES = 20
    n = len(points)
    dp = defaultdict(lambda: float('inf'))
    parent = dict()  # key: (l, r, i)，value: (prev_l, prev_r, 'L' or 'R')
    dp[(-1, -1, 0, 0)] = 0

    for i in range(n):
        new_dp = defaultdict(lambda: float('inf'))
        new_parent = dict()
        p_i = points[i]
        # print(len(dp))

        for (l, r, l_block, r_block), cost in dp.items():
            # 左手可用
            if i >= l_block:
                if l == -1:
                    left_burst = 0
                else:
                    p_l = points[l]
                    left_burst = compute_burst(p_l.x, p_i.x, p_l.t, p_i.t, "L", a, type1=p_l.note_type)
                new_cost = cost + left_burst
                key = (i, r, p_i.block_id, r_block)
                if new_cost < new_dp[key]:
                    new_dp[key] = new_cost
                    new_parent[(i, r, i)] = (l, r, 'L')

            # 右手可用
            if i >= r_block:
                if r == -1:
                    right_burst = 0
                else:
                    p_r = points[r]
                    right_burst = compute_burst(p_r.x, p_i.x, p_r.t, p_i.t, "R", a, type1=p_r.note_type)
                new_cost = cost + right_burst
                key = (l, i, l_block, p_i.block_id)
                if new_cost < new_dp[key]:
                    new_dp[key] = new_cost
                    new_parent[(l, i, i)] = (l, r, 'R')

            # 左右都不能用（3壓）
            if i < l_block and i < r_block:
                if not is_three_hand:
                    is_three_hand = True
                key = (l, r, l_block, r_block)
                if cost < new_dp[key]:  # 沒增加 burst
                    new_dp[key] = cost
                    new_parent[(l, r, i)] = (l, r, '3+')

        # 剪枝，讓DP快一點，大概不影響
        if len(new_dp) > MAX_STATES:
            sorted_states = sorted(new_dp.items(), key=lambda item: item[1])
            limited_states = sorted_states[:MAX_STATES]

            new_dp = defaultdict(lambda: float('inf'))
            for key, cost in limited_states:
                new_dp[key] = cost
        dp = new_dp
        parent.update(new_parent)

    # 找出最小總爆發值以及對應終點狀態
    min_cost = float('inf')
    final_state = None
    for (l, r, l_block, r_block), cost in dp.items():
        if cost < min_cost:
            min_cost = cost
            final_state = (l, r)

    # 回溯路徑
    hand_sequence = [''] * n
    cur_l, cur_r = final_state
    for i in reversed(range(n)):
        prev_l, prev_r, hand = parent[(cur_l, cur_r, i)]
        hand_sequence[i] = hand
        if hand == 'L':
            cur_l = prev_l
        elif hand == 'R':
            cur_r = prev_r

    return min_cost, hand_sequence
    # min_cost其實後來沒在管，反正要算burst序列
    # hand_sequence是R或L

def calculate_burst(note_list, tempo_list, is_debug=False):
    Point = namedtuple('Point', ['idx', 't', 'x', 'block_id', 'dur', 'pid', 'tick', 'note_type'])
    points = []
    # 除了drag child的note都視為click
    for note in note_list:
        if note["type"] == 7 or note["type"] == 4:
            continue
        t = tick_to_seconds_precise(note["tick"], tempo_list)
        if note["type"] == 1 or note["type"] == 2: # 加上hold手不能動的時間
            hold_length = tick_to_seconds_precise(note["tick"] + note["hold_tick"], tempo_list) - t
            points.append((t, note["x"], hold_length, note["page_index"], note["tick"], note["type"]))
        else:
            points.append((t, note["x"], 0, note["page_index"], note["tick"], note["type"]))

    N = len(points)
    # 做一點轉換
    new_points = []
    for i, (t, x, dur, pid, tick, note_type) in enumerate(points):
        if dur == 0:
            dur = 0.03 # 把所有click都設定最小間隔，可以偵測三壓
        t_block = t + dur
        i_cur = i + 1
        while i_cur < N and points[i_cur][0] <= t_block:
            i_cur += 1
        new_points.append(Point(i, t, x, i_cur, dur, pid, tick, note_type))

    points = new_points

    if is_debug:
        print(f"原物量：{len(note_list)}, 處理後物量：{len(points)}")

    # 這個函數是在抽取譜面特徵時的最大瓶頸
    start_time = time.time()
    cost, hands = minimize_burst_with_path(points)
    end_time = time.time()
    # print(f"執行時間：{end_time - start_time:.6f} 秒")

    burst_values = []
    prev_L = -1
    prev_R = -1
    # 重算一次burst序列
    for i, (p, hand) in enumerate(zip(points, hands)):
        if hand == 'L':
            if prev_L == -1:
                burst = 0
            else:
                p_prev = points[prev_L]
                burst = compute_burst(p_prev.x, p.x, p_prev.t, p.t, 'L', a=0.2, is_debug=is_debug, is_post_process=True)
            prev_L = i
        elif hand == 'R':
            if prev_R == -1:
                burst = 0
            else:
                p_prev = points[prev_R]
                burst = compute_burst(p_prev.x, p.x, p_prev.t, p.t, 'R', a=0.2, is_debug=is_debug, is_post_process=True)
            prev_R = i
        else:
            burst = 0  # 'error' 或 '3+' 設為0

        burst_values.append(burst)

    if is_debug:
        # 一些基本資訊
        print(f"執行時間：{end_time - start_time:.6f} 秒")
        print("最小爆發總和（最佳左右手分配）：", sum(burst_values))
        print("左右手分配：", hands)

        colors = {'L': 'blue', 'R': 'red', '3+': 'yellow'}
        # burst分布直方圖
        plt.figure(figsize=(8, 6))
        plt.hist(burst_values, bins=30, color='skyblue', edgecolor='black')
        plt.xlabel("burst_values")
        plt.ylabel("appear time")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    if is_debug and True:
        # 手順圖
        plt.figure(figsize=(8, 400))

        for i, (p, hand) in enumerate(zip(points, hands)):
            color = colors.get(hand, 'gray')  # 預設為 gray，避免意外值
            label = hand if hands[:i].count(hand) == 0 else ""

            plt.scatter(p.x, p.t, color=color, s=100, label=label)
            plt.text(p.x, p.t, f' {i}', verticalalignment='center', horizontalalignment='center')

            if p.dur > 0.031:
                plt.plot([p.x, p.x], [p.t, p.t + p.dur], color=color, linewidth=4, alpha=0.5)

        plt.legend()
        plt.grid(True)
        plt.xlabel("X 位置")
        plt.ylabel("時間（秒）")

        plt.xlabel('Position (X)')
        plt.ylabel('Time (t)')
        plt.gca().invert_yaxis()  # 時間向下遞增，視覺上從上到下順序
        plt.legend()
        plt.grid(True)
        plt.show()

    return burst_values, hands, points

# 測試，理論上重新讀檔會慢一點，但少量測試應該沒差
# with open("Cytus2Chart/" + "neko002_010+chaos.json", newline='') as f:
# with open("Cytus2Chart/" + "xenon001_007+glitch.json", newline='') as f:
# with open("Cytus2Chart/" + "neko001_061+chaos.json", newline='') as f:
chart_json = all_chart_json[324]
chart_json = all_chart_json[416] # Lucid Traveler
chart_json = all_chart_json[211] # iL
chart_json = all_chart_json[182] # Halloween Party



# chart_json = all_chart_json[443]
note_list = chart_json["note_list"]
note_list = sorted(note_list, key=lambda x: x['tick'])
tempo_list = chart_json["tempo_list"]

start_time = time.time()
burst_values, hands, points = calculate_burst(note_list, tempo_list, is_debug = True)
end_time = time.time()
print(f"執行時間：{end_time - start_time:.6f} 秒")


In [ ]:
# 用公式算節奏複雜度
import itertools

def rhythm_entropy(events, threshold_time=960, is_page=False, is_debug=False):
    """
    events: 節奏事件序列（可為音符時值列表）
    返回：節奏熵值（bit為單位）
    """
    events = events[events < threshold_time]
    events = events[events > 0]
    if len(events) == 0:
        return 0, 0
    events = np.round(events, 5)

    # 原本用後一個的兩倍是否等於前一個來偵測swing，但沒用
    # 後來改為偵測時值變化次數，不多
    swing_beat = 0
    for i in range(len(events)-2):
        if events[i] != events[i+1]:
            swing_beat += 1
        if events[i] != events[i+2]:
            swing_beat += 0.2
    swing_beat_ratio = swing_beat/(len(events)-1)

    # 每種時值出現次數
    unique, counts = np.unique(events, return_counts=True)

    # 同時過濾 unique 和 counts
    # count太小當成zure就不要了
    mask = (unique != 0) & (counts > counts.sum()*0.01)
    unique = unique[mask]
    counts = counts[mask]

    probs = counts / counts.sum()

    ### 計算音符時值的 集中度，越不集中數值越高，表示節奏越複雜
    # 計算熵
    entropy = -np.sum(probs * np.log2(probs))
    if is_page: # 應該沒用到
        entropy *= np.log2(len(events))

    # 這段觀察不同時值之間是否可以整除，作為複雜度參考，效果不好
    unique = np.concatenate([unique, [960]])
    new_counts = max(counts.sum() // 10, 2)
    counts = np.concatenate([counts, [new_counts]])

    probs = counts / counts.sum()
    ratio_penalty = 0
    total_weight = 0
    if len(unique) >= 2:
        for (i, a), (j, b) in itertools.combinations(enumerate(unique), 2):
            ratio = max(a, b) / min(a, b)
            ratio = round(ratio, 2)

            weight = probs[i] * probs[j]
            total_weight += weight

            # 整除（比值是整數）的情況，複雜度低
            if ratio.is_integer():
                continue

            # 2:3, 3:5, 5:6 等更難整除 → 複雜度較高
            ratio_penalty += weight  # 可以改成 weight * ratio 或更複雜公式

        ratio_penalty /= total_weight

    if is_debug:
        print(ratio_penalty, unique, counts)

    return entropy, ratio_penalty, swing_beat_ratio

def get_rhythm_entropy(note_list, page_scores, points, hands, page_list_length):
    # 這個函數只是把相關計算前處理封裝起來而已
    page_scores_sq = np.zeros(page_list_length)
    for page_index, score in page_scores.items():
        page_scores_sq[page_index] = score
    page_scores_mid = np.percentile(page_scores_sq, 75)

    tick_list = []
    R_tick_list = []
    L_tick_list = []

    for i, p in enumerate(points):
        if hands[i] == "R":
            R_tick_list.append(p.tick)
        else:
            L_tick_list.append(p.tick)
        tick_list.append(p.tick)

    tick_list = np.array(tick_list)
    tick_list = np.diff(tick_list)
    R_tick_list = np.array(R_tick_list)
    R_tick_list = np.diff(R_tick_list)
    L_tick_list = np.array(L_tick_list)
    L_tick_list = np.diff(L_tick_list)

    # 分多段計算，避免有些歌只有一半是跳拍被稀釋掉了
    bin_count = 9
    all_bins = np.round(np.linspace(0, len(tick_list)+1, bin_count+1)).astype(int)
    R_bins = np.round(np.linspace(0, len(R_tick_list)+1, bin_count+1)).astype(int)
    L_bins = np.round(np.linspace(0, len(L_tick_list)+1, bin_count+1)).astype(int)

    test_entropy_list = []
    test_ratio_penalty_list = []
    swing_beat_ratio_list = []

    for b in range(bin_count):
        all_diff = tick_list[all_bins[b]:all_bins[b+1]]
        R_diff = R_tick_list[R_bins[b]:R_bins[b+1]]
        L_diff = L_tick_list[L_bins[b]:L_bins[b+1]]
        # 原本嘗試跟左右手分別的節奏合併後再放進去，但效果不佳
        # all_diff = np.concatenate([all_diff, R_diff, L_diff])
        # all_diff = np.concatenate([R_diff, L_diff])
        test_entropy, test_ratio_penalty, swing_beat_ratio = rhythm_entropy(all_diff, is_debug=False)
        test_entropy_list.append(test_entropy)
        test_ratio_penalty_list.append(test_ratio_penalty)
        swing_beat_ratio_list.append(swing_beat_ratio)

    # 取最難的80%位置
    test_entropy = np.percentile(test_entropy_list, 80)
    test_ratio_penalty = np.percentile(test_ratio_penalty_list, 80)
    swing_beat_ratio = np.mean(swing_beat_ratio_list)

    return test_entropy, test_ratio_penalty, swing_beat_ratio


test_id = [442, 567, 453, 315, 563, 408, 261, 365, 560, 32, 277, 502, 211, 561, 438, 569, 217, 103, 306, 500, 523, 9, 137, 267]
# test_id = []
# test_id = list(range(642))

for c_id in test_id:
    with open("Cytus2Chart/" + available_chart[c_id], newline='') as f:
        chart_json = json.load(f)
    ### 一些list
    tempo_list = chart_json["tempo_list"]
    tempo_list = sorted(tempo_list, key=lambda x: x['tick'])
    page_list = chart_json["page_list"]
    page_list = sorted(page_list, key=lambda x: x['start_tick'])
    note_list = chart_json["note_list"]
    note_list = sorted(note_list, key=lambda x: x['tick'])

    page_scores = get_page_score(note_list, page_list)
    burst_values, hands, points = calculate_burst(note_list, tempo_list)
    test_entropy, test_ratio_penalty, swing_beat_ratio = get_rhythm_entropy(note_list, page_scores, points, hands, len(page_list))

    print(f"{all_song_name[c_id][:30]:<32}\t{round(test_entropy, 2)}\t{round(test_ratio_penalty, 2)}\t{round(swing_beat_ratio, 2)}")


In [9]:
# 土法煉鋼判斷複雜節奏，效果比前面那坨什麼熵的還好一點
def get_complex_score(tap_tick_list, page_list):

    n = len(tap_tick_list)
    if n == 0:
        return 0, 0
    m = len(page_list)
    complex_beat_count = 0
    l_ptr = 0  # 維護480 tick以內的範圍，不用每次都遍歷全部的tap_tick_list
    r_ptr = 0
    type_count = dict()
    cur_pid = 0  # 用來確定現在的note的page位置
    cur_page_start = 0
    for i in range(n):
        tick = tap_tick_list[i]
        while page_list[cur_pid]["end_tick"] < tick:
            cur_pid += 1
            cur_page_start = page_list[cur_pid]["start_tick"]
        while l_ptr < n and tap_tick_list[l_ptr] < tick - 480:
            l_ptr += 1
        while r_ptr < n and tap_tick_list[r_ptr] <= tick + 480:
            r_ptr += 1
        nearby = tap_tick_list[l_ptr:r_ptr]
        tick_shift = (cur_page_start - tick) % 480  # 強制從頁邊起算第一拍
        tick_shift = round(tick_shift / 10, 0) * 10 % 480

        if tick_shift == 0:
            continue

        elif tick_shift == 60:  ## 1/8
            type_count[0] = type_count.get(0, 0) + 1
            unit = 60
            if tick - unit in nearby and tick + unit in nearby:  # 32分交互
                complex_beat_count += 0.05
                continue
            if tick - unit in nearby:  # 拍頭偽雙壓
                complex_beat_count += 1
                continue
            complex_beat_count += 2.5

        elif tick_shift == 80:  ## 1/6
            type_count[1] = type_count.get(1, 0) + 1
            unit = 80
            if tick - unit in nearby and tick + unit in nearby:  # 24分交互
                complex_beat_count += 0.05
                continue
            if tick - unit in nearby:  # 拍頭偽雙壓
                complex_beat_count += 1
                continue
            complex_beat_count += 2.5

        elif tick_shift == 120:  ## 1/4
            type_count[2] = type_count.get(2, 0) + 1
            unit = 120
            if tick - unit in nearby and tick + unit in nearby:  # 16分交互
                continue
            if tick - unit in nearby or tick + unit in nearby:  # 二連點之類
                complex_beat_count += 0.5
                continue
            complex_beat_count += 2  # 純1/4，搞節奏心態

        elif tick_shift == 160:  ## 1/3
            type_count[3] = type_count.get(3, 0) + 1
            unit = 160
            if tick - unit in nearby and tick + unit in nearby:  # 12分交互
                complex_beat_count += 0.05
                continue
            if tick - 80 in nearby and tick + 80 in nearby:  # 24分交互
                complex_beat_count += 0.05
                continue
            if tick - 80 not in nearby and tick + 80 in nearby:  # 特別難搞的小swing節奏
                complex_beat_count += 3
                continue
            if tick - unit not in nearby and tick + unit in nearby:  # XOO XOO
                complex_beat_count += 1.5
                continue
            if tick - unit in nearby and tick + unit not in nearby:  # OOX OOX 可能沒這麼怪
                complex_beat_count += 0.8
                continue
            complex_beat_count += 1  # 其他單出十二分音的狀況

        elif tick_shift == 180:  ## 3/8
            type_count[4] = type_count.get(4, 0) + 1
            unit = 60
            if tick - unit in nearby and tick + unit in nearby:  # 32分交互
                complex_beat_count += 0.1
                continue
            complex_beat_count += 1  # 不是交互出現都很怪

        elif tick_shift == 240:  ## 1/2
            type_count[5] = type_count.get(5, 0) + 1
            unit = 240
            if tick - unit not in nearby and tick + unit not in nearby:  # 反半拍
                complex_beat_count += 0.1
                continue

        elif tick_shift == 300:  ## 5/8
            type_count[6] = type_count.get(6, 0) + 1
            unit = 60
            if tick - unit in nearby and tick + unit in nearby:  # 32分交互
                complex_beat_count += 0.1
                continue
            complex_beat_count += 1  # 不是交互出現都很怪

        elif tick_shift == 320:  ## 2/3
            type_count[7] = type_count.get(7, 0) + 1
            unit = 160
            if tick - unit not in nearby and tick + unit in nearby:  # 大swing
                complex_beat_count += 0.5
                continue
            if tick - 60 in nearby and tick + 60 not in nearby:  # 1/4 還能偽雙？
                complex_beat_count += 1.5
                continue
            # 原則上都出現過了

        elif tick_shift == 360:  ## 3/4
            type_count[8] = type_count.get(8, 0) + 1
            unit = 120
            if tick - unit in nearby and tick + unit in nearby:  # 16分交互
                continue
            if tick - unit in nearby or tick + unit in nearby:  # 二連點之類
                complex_beat_count += 0.5
                continue
            if tick - 360 in nearby and tick - 240 not in nearby:
                continue  # 有可能是332節奏的一部分
            complex_beat_count += 2

        elif tick_shift == 400:  ## 5/6
            type_count[9] = type_count.get(9, 0) + 1
            unit = 80
            if tick - unit in nearby and tick + unit in nearby:  # 24分交互
                complex_beat_count += 0.05
                continue
            if tick + unit in nearby:  # 小swing
                complex_beat_count += 3
                continue
            complex_beat_count += 2

        elif tick_shift == 420:  ## 7/8
            type_count[10] = type_count.get(10, 0) + 1
            unit = 60
            if tick - unit in nearby and tick + unit in nearby:  #32分交互
                complex_beat_count += 0.05
                continue
            if tick + unit in nearby:  # 拍尾偽雙壓
                complex_beat_count += 1
                continue
            complex_beat_count += 2.5

        else:  # 持續性擺歪？
            type_count[11] = type_count.get(11, 0) + 1
            complex_beat_count += 1.5

    # print(type_count)
    return complex_beat_count, complex_beat_count / n


test_list = [569]
for c_id in test_list:
    chart_json = all_chart_json[c_id]
    note_list = chart_json["note_list"]
    note_list = sorted(note_list, key=lambda x: x['tick'])
    page_list = chart_json["page_list"]
    page_list = sorted(page_list, key=lambda x: x['start_tick'])
    tap_tick_list = set()
    for note in note_list:
        if note["type"] == 7 or note["type"] == 4:
            continue
        tap_tick_list.add(note["tick"])
    tap_tick_list = list(tap_tick_list)
    tap_tick_list = sorted(tap_tick_list)
    complex_beat_count, complex_beat_ratio = get_complex_score(tap_tick_list, page_list)

    print(f"{c_id}\t{pad_display(all_song_name[c_id][:25], 32)}\t{round(complex_beat_count, 2)}\t{round(complex_beat_ratio, 2)}")
    # print()


569	Take me to the Future           	87.15	0.13


In [10]:
# 譜面特徵抽取
import heapq

# 轉換資料
Y = np.array([real_diff_list]).reshape(-1)
# 抽取特徵

is_test_time = True
is_test_time = False # 時間分析用

test_list = [495, 368, 355, 19, 127, 442, 569, 111, 408, 619, 137, 24, 400]

def get_all_feature(all_chart_json):
    X = []
    for i_c, chart_json in enumerate(tqdm(all_chart_json, desc="Processing charts")):
        # for i_c in test_list:
        #   chart_json = all_chart_json[i_c]

        feature = []
        feature_name = []


        ### 一些list
        if is_test_time:
            start_time = time.time()
        # 因為轉換的關係所以不一定有sort好
        tempo_list = chart_json["tempo_list"]
        tempo_list = sorted(tempo_list, key=lambda x: x['tick'])
        page_list = chart_json["page_list"]
        page_list = sorted(page_list, key=lambda x: x['start_tick'])
        note_list = chart_json["note_list"]
        note_list = sorted(note_list, key=lambda x: x['tick'])
        if is_test_time:
            end_time = time.time()
            print("Sort list")
            print(f"執行時間：{end_time - start_time:.6f} 秒")


        if is_test_time:
            start_time = time.time()
        ### SONG_LENGTH
        last_tick = max([n["tick"]+n["hold_tick"] for n in chart_json["note_list"]])
        song_length = tick_to_seconds_precise(last_tick, tempo_list) - 200
        if song_length > 0:
            song_length = song_length**0.7 # 不太有歌超過3分鐘
        SONG_LENGTH = song_length + 200
        feature.append(SONG_LENGTH)
        feature_name.append("SONG_LENGTH")
        if is_test_time:
            end_time = time.time()
            print("SONG_LENGTH")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        if is_test_time:
            start_time = time.time()

        ### AVG_BPM
        # AVG_BPM = last_tick/8/SONG_LENGTH
        # feature.append(AVG_BPM)
        # feature_name.append("AVG_BPM")

        ### MAIN_BPM
        max_duration = 0
        for i in range(len(tempo_list)):
            start_tick = tempo_list[i]['tick']
            value = tempo_list[i]['value']  # μs/beat
            end_tick_segment = last_tick if i == len(tempo_list) - 1 else tempo_list[i + 1]['tick']

            duration = (end_tick_segment - start_tick) * value

            if duration > max_duration:
                max_duration = duration
                MAIN_BPM = 60_000_000 / value  # 計算BPM
        feature.append(MAIN_BPM)
        feature_name.append("MAIN_BPM")
        if is_test_time:
            end_time = time.time()
            print("BPM")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        ### NOTE_TYPE_COUNT
        # 有一堆指標跟這個有相關，所以這部分效果相比之下沒這麼好
        # 如果測試資料把hard放進來，模型會用有沒有flick來作弊
        # Type 0: Click
        # Type 1: Hold
        # Type 2: L-Hold
        # Type 3: Drag-head
        # Type 4: Drag-child
        # Type 5: Flick
        # Type 6: CDrag-head
        # Type 7: CDrag-child
        type_counter = Counter(note['type'] for note in note_list)
        type_counts = [type_counter.get(i, 0) for i in [3, 4, 5, 6, 7]] # 沒有考慮drag的指標
        feature.extend(type_counts)
        # feature_name.extend(["Click", "Hold", "L-Hold", "Drag-head", "Drag-child", "Flick", "CDrag-head", "CDrag-child"])
        feature_name.extend(["Drag-head", "Drag-child", "Flick", "CDrag-head", "CDrag-child"])

        if is_test_time:
            start_time = time.time()
        # 算每頁加權分數
        page_scores = get_page_score(note_list, page_list)
        if is_test_time:
            end_time = time.time()
            print("Get Page Score")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        if is_test_time:
            start_time = time.time()
        # 每頁加權分數、用頁長考慮時間密度
        # 跟很多指標同質性太高所以刪掉
        # page_score_time_density = []
        # for page_index, score in page_scores.items():
        #   if score == 0:
        #     continue
        #   page = page_list[page_index]
        #   page_length = (page['end_tick'] - page['start_tick']) / 480
        #   if page_length < 0.01:
        #     continue
        #   page_score_time_density.append(score / page_length)
        # max_score = max(page_score_time_density)
        # median_score = np.median(page_score_time_density)
        # p75_score = np.percentile(page_score_time_density, 75)
        # p90_score = np.percentile(page_score_time_density, 90)

        # feature.append(max_score)
        # feature.append(median_score)
        # feature.append(p75_score)
        # feature.append(p90_score)
        # feature_name.append("page_time_max_score")
        # feature_name.append("page_time_median_score")
        # feature_name.append("page_time_p75_score")
        # feature_name.append("page_time_p90_score")

        # 不考慮頁長，只考慮視覺上的密度
        page_scores_list = list(page_scores.values())
        page_scores_list = [score for score in page_scores_list if score != 0]
        # max_score = max(page_scores_list)
        third_score = heapq.nlargest(3, page_scores_list)[2 if len(page_scores_list) >= 3 else -1]
        # page_space_median_score = np.median(page_scores_list)
        page_space_p90_score = np.percentile(page_scores_list, 90)
        # page_space_p75_score = np.percentile(page_scores_list, 75)

        # feature.append(max_score)
        feature.append(third_score)
        # feature.append(page_space_median_score)
        # feature.append(page_space_p75_score)
        feature.append(page_space_p90_score)

        # feature_name.append("page_space_max_score")
        feature_name.append("page_space_third_score")
        # feature_name.append("page_space_median_score")
        # feature_name.append("page_space_p75_score")
        feature_name.append("page_space_p90_score")

        # 每頁耐力
        # 用數頁的最小值表示
        N = max(page_scores.keys()) + 1
        page_score_sequence = [page_scores.get(i, 0) for i in range(N)]
        window_sizes = [8, 16, 24, 32]
        results = {}
        for window_size in window_sizes:
            max_min = float('-inf')
            for i in range(len(page_score_sequence) - window_size + 1):
                window = page_score_sequence[i:i+window_size]
                local_min = min(window)
                if local_min > max_min:
                    max_min = local_min
            results[window_size] = max_min
        # feature.append(results[8])
        # feature.append(results[16])
        feature.append(results[24])
        # feature.append(results[32])
        # feature_name.extend(["page_endurance_8"])
        # feature_name.extend(["page_endurance_16"])
        feature_name.extend(["page_endurance_24"])
        # feature_name.extend(["page_endurance_32"])
        if is_test_time:
            end_time = time.time()
            print("Page Score Utility")
            print(f"執行時間：{end_time - start_time:.6f} 秒")


        ### 拍號、頁長
        # 效果不彰，很多地方都簡化掉了
        if is_test_time:
            start_time = time.time()
        page_ticks = []
        for page in page_list:
            ticks = page['end_tick'] - page['start_tick']
            page_ticks.append(ticks)
        # 1. 統計 tick 出現次數
        counter = Counter(page_ticks)
        most_common = counter.most_common()
        # 第一常見 tick
        most_common_tick, most_common_count = most_common[0]
        if most_common_tick in [960, 480]:
            time_signatures_score = 0
        elif most_common_tick in [1920, 1440, 720]:
            time_signatures_score = 10
        elif most_common_tick < 1200:
            time_signatures_score = 15
        else:
            time_signatures_score = 25
        feature.append(time_signatures_score)
        feature_name.append("time_signatures_score")

        # 2. 第一常見佔比
        total = len(page_ticks)
        most_common_ratio = most_common_count / total
        # feature.append(most_common_ratio)
        # feature_name.append("most_common_tick_ratio")

        # 6. 變化次數（相鄰值不一樣就算變化）
        changes = sum(1 for i in range(1, total) if page_ticks[i] != page_ticks[i-1])
        feature.append(changes)
        feature_name.append("page_tick_changes")

        # 7. 不同 tick 數值的種類數
        # unique_ticks_count = len(counter)
        # feature.append(unique_ticks_count)
        # feature_name.append("unique_ticks_count")
        if is_test_time:
            end_time = time.time()
            print("Time Signatures Score")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        ### 爆發度、三壓
        if is_test_time:
            start_time = time.time()
        burst_values, hands, points = calculate_burst(note_list, tempo_list)
        # 神奇的自動分配手順咚咚
        # 跟burst相關的指標表現都很好
        if is_test_time:
            end_time = time.time()
            print("Get Burst")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        if is_test_time:
            start_time = time.time()
        # print(len(burst_values), len(hands), len(points))
        THREE_HAND = hands.count("3+")
        feature.append(THREE_HAND)
        feature_name.append("THREE_HAND") # 沒什麼用

        burst_array = np.array(burst_values)
        sorted_burst = np.sort(burst_array)[::-1]
        n = len(burst_values)
        max_val = sorted_burst[0]
        median_val = sorted_burst[int(0.5 * n)]
        fifth_val = sorted_burst[4]
        n = len(sorted_burst)
        p90_val = sorted_burst[int((1 - 0.9) * n)]
        p75_val = sorted_burst[int((1 - 0.75) * n)]
        burst_song_avg = np.sum(sorted_burst[int(0.1 * n):int(0.9 * n)]) / SONG_LENGTH
        # feature.append(max_val)
        # feature.append(median_val)
        feature.append(fifth_val)
        feature.append(p90_val)
        # feature.append(p75_val)
        feature.append(burst_song_avg)
        # feature_name.append("burst_max")
        # feature_name.append("burst_median")
        feature_name.append("burst_fifth")
        feature_name.append("burst_p90")
        # feature_name.append("burst_p75")
        feature_name.append("burst_song_avg")

        if is_test_time:
            end_time = time.time()
            print("Burst Rank")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        if is_test_time:
            start_time = time.time()

        ## 用爆發值算耐力
        burst_series = pd.Series(burst_array)
        results = {}
        for window_size in [8, 100]:
            # 使用 rolling() 搭配 quantile(0.25)
            local_25 = burst_series.rolling(window_size).quantile(0.25)
            max_local_25 = local_25.max()
            results[window_size] = max_local_25

        feature.append(results[8])
        # feature.append(results[16])
        # feature.append(results[32])
        # feature.append(results[64])
        feature.append(results[100]) # 大約100 combo
        feature_name.append("burst_endurance_8")
        # feature_name.append("burst_endurance_16")
        # feature_name.append("burst_endurance_32")
        # feature_name.append("burst_endurance_64")
        feature_name.append("burst_endurance_100")
        if is_test_time:
            end_time = time.time()
            print("Burst Endurance")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        if is_test_time:
            start_time = time.time()

        # 考慮雙手同時出力可能比較難
        l_ptr = 0
        L_burst = 0
        R_burst = 0
        LR_low_burst_list = []
        window = 1
        for r_ptr, p in enumerate(points):
            if hands[r_ptr] == "L":
                L_burst += burst_values[r_ptr]
            elif hands[r_ptr] == "R":
                R_burst += burst_values[r_ptr]

            while l_ptr < r_ptr and points[l_ptr].t < p.t - window:
                if hands[l_ptr] == "L":
                    L_burst -= burst_values[l_ptr]
                elif hands[l_ptr] == "R":
                    R_burst -= burst_values[l_ptr]
                l_ptr += 1
            LR_low_burst_list.append(min(L_burst, R_burst))

        LR_low_burst_array = np.array(LR_low_burst_list)
        # burst_LR_low_75 = np.percentile(LR_low_burst_array, 75)
        # burst_LR_low_80 = np.percentile(LR_low_burst_array, 80)
        burst_LR_low_max = max(LR_low_burst_array[len(LR_low_burst_array) * 4 // 5:])
        # feature.append(burst_LR_low_75)
        # feature.append(burst_LR_low_80)
        feature.append(burst_LR_low_max)
        # feature_name.append("burst_LR_low_75")
        # feature_name.append("burst_LR_low_80")
        feature_name.append("burst_LR_low_max")

        if is_test_time:
            end_time = time.time()
            print("Burst LR Low")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        if is_test_time:
            start_time = time.time()
        L_last = 0
        R_last = 0
        L_double_time = []
        R_double_time = []
        all_double_time = []
        double_click_threshold = 70 if most_common_tick < 840 else 140

        n = len(points)
        for id in range(n):
            if hands[id] == "L":
                if points[id].tick - points[L_last].tick < double_click_threshold:
                    L_double_time.append(points[id].t)
                    all_double_time.append(points[id].t)
                L_last = id
            elif hands[id] == "R":
                if points[id].tick - points[R_last].tick < double_click_threshold:
                    R_double_time.append(points[id].t)
                    all_double_time.append(points[id].t)
                R_last = id

        # window_sizes = [1, 1.5, 2, 2.5, 3]
        # results = {}
        # for window_size in window_sizes:
        #     w_start = 0
        #     results.setdefault(window_size, 0)
        #     for i in range(len(all_double_time)):
        #         while all_double_time[i] - window_size > all_double_time[w_start]:
        #             w_start += 1
        #         results[window_size] = max(results[window_size], i - w_start)

        # feature.append(results[1])
        # feature.append(results[1.5])
        # feature.append(results[2])
        # feature.append(results[2.5])
        # feature.append(results[3])
        # feature_name.append("double_click_1")
        # feature_name.append("double_click_1.5")
        # feature_name.append("double_click_2")
        # feature_name.append("double_click_2.5")
        # feature_name.append("double_click_3")
        feature.append(len(all_double_time))
        feature_name.append("double_count")
        if is_test_time:
            end_time = time.time()
            print("Double click")
            print(f"執行時間：{end_time - start_time:.6f} 秒")

        # 節奏相關指標
        if is_test_time:
            start_time = time.time()
        all_rhythm_entropy, rhythm_ratio_penalty, swing_beat_ratio = get_rhythm_entropy(note_list, page_scores, points, hands, len(page_list))
        feature.append(all_rhythm_entropy)
        # feature.append(rhythm_ratio_penalty)
        feature.append(swing_beat_ratio)
        feature_name.append("all_rhythm_entropy")
        # feature_name.append("rhythm_ratio_penalty")
        feature_name.append("swing_beat_ratio")
        if is_test_time:
            end_time = time.time()
            print("Rhythm Entropy")
            print(f"執行時間：{end_time - start_time:.6f} 秒")


        if is_test_time:
            start_time = time.time()
        ### 節拍位置
        beat_type = []
        tap_tick_list = set()
        for note in note_list:
            if note["type"] == 7 or note["type"] == 4:
                continue
            tap_tick_list.add(note["tick"])
        tap_tick_list = list(tap_tick_list)
        tap_tick_list = sorted(tap_tick_list)

        complex_beat_count, complex_beat_ratio = get_complex_score(tap_tick_list, page_list)

        feature.append(complex_beat_count)
        # feature.append(complex_beat_ratio)
        feature_name.append("complex_beat_count")
        # feature_name.append("complex_beat_ratio")
        # print(f"{round(complex_beat_count, 2)}\t{round(complex_beat_ratio, 2)}")
        if is_test_time:
            end_time = time.time()
            print("Complex Beat")
            print(f"執行時間：{end_time - start_time:.6f} 秒")



        # print(feature)
        X.append(feature)
        # if i_c > -1:
        #   break

    return X, feature_name

X, feature_name = get_all_feature(all_chart_json)
X = np.array(X)
# print(X)
print()
print(X.shape, Y.shape, len(feature_name))


Processing charts: 100%|██████████| 643/643 [00:20<00:00, 31.56it/s]


(643, 23) (643,) 23


In [11]:
print(X.shape, Y.shape, len(feature_name))

(643, 23) (643,) 23


In [12]:
# 投票模型
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# 模型融合怪，要挑一下
# train R2 比較高，但test不一定
def getVoteModel():
  # 建立基模型
  lr = LinearRegression()
  ridge = Ridge()
  lasso = Lasso()
  dt = DecisionTreeRegressor(random_state=42)
  rf = RandomForestRegressor(n_estimators=40, random_state=42)
  gbr = GradientBoostingRegressor(n_estimators=40, random_state=42)
  svr = SVR()
  ada = AdaBoostRegressor(n_estimators=20, random_state=42)
  et = ExtraTreesRegressor(n_estimators=40, random_state=42)
  mlp = MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=500, random_state=42)

  # 組合 Voting Regressor
  voting_reg = VotingRegressor(
      estimators=[
          # ('lr', lr),
          # ('ridge', ridge),
          # ('lasso', lasso),
          ('dt', dt),
          ('rf', rf),
          # ('gbr', gbr),
          # ('svr', svr),
          # ('ada', ada),
          ('et', et),
          # ('mlp', mlp)
      ]
  )
  return voting_reg

In [14]:
# 訓練
# from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

candidate = [
    'burst_p90',
    'page_space_p90_score',
    'page_space_third_score',
    'burst_song_avg',
    'complex_beat_count',
    'Drag-child',
    'burst_LR_low_max',
    'double_count',
    'SONG_LENGTH',
    'burst_endurance_8',
    'CDrag-head',
    'MAIN_BPM',
    'burst_fifth'
]
feat2idx = {f: i for i, f in enumerate(feature_name)}
idxs = [feat2idx[feat] for feat in candidate]
X_subset = X[:, idxs]

# 1. 分割訓練與測試集

used_data_size = None
indices = np.arange(len(X[:used_data_size]))
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, Y, indices, test_size=0.1, random_state=75
)
# X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
#     X_subset, Y, indices, test_size=0.1, random_state=75
# )
# 標準化，不一定好
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# 降低維度，不一定好
# pca = PCA(n_components=0.98)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

print(f"Train size:{len(X_train)}")
print(f"Test size:{len(X_test)}")

# model = RandomForestRegressor(n_estimators=100, random_state=42)
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
# model = getVoteModel()

model.fit(X_train, y_train)


# model = TabNetRegressor(optimizer_params=dict(lr=3e-2), verbose=0)
# y_train = np.reshape(y_train, (-1, 1))
# y_test = np.reshape(y_test, (-1, 1))
# model.fit(
#     X_train, y_train,
#     eval_set=[(X_test, y_test)],
#     eval_metric=['mse'],
#     max_epochs=3000,
#     patience=100,
#     batch_size=1024,
#     virtual_batch_size=128,
#     num_workers=0,
#     augmentations=augmentations,
#     drop_last=False
# )

y_pred = model.predict(X_test)
y_test = y_test.reshape(-1)
y_pred = y_pred.reshape(-1)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

data = []
for i, idx in enumerate(test_idx):
    song_name = all_song_name[idx]
    diff_name = available_chart[idx].split('+')[1][:-5]
    data.append((idx, song_name, diff_name, y_test[i], round(y_pred[i], 2), round(y_pred[i] - y_test[i], 2)))

SORT_BY = 5
# SORT_BY = 4

data = sorted(data, key=lambda x: (x[SORT_BY]))
print()
print(f"ID\t{pad_display('曲名', 32)}\t難度\t等級\t預測\t誤差")
for idx, s, d, t, p, diff in data:
    print(f"{idx}\t{pad_display(s[:30], 35)}\t{d}\t{t}\t{p}\t{diff}")

# y_pred = np.round(y_pred, 2)
# print("\t".join(map(str, np.reshape(y_pred, (-1,)))))
# print("\t".join(map(str, np.reshape(y_test, (-1,)))))

print()
print("TEST Mean Squared Error:", mse)
print("TEST R² Score:", r2)
print("===================================================================")

if False:
    y_pred = model.predict(X_train)
    y_train = y_train.reshape(-1)
    y_pred = y_pred.reshape(-1)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)

    data = []
    for i, idx in enumerate(train_idx):
        song_name = all_song_name[idx]
        diff_name = available_chart[idx].split('+')[1][:-5]
        data.append((idx, song_name, diff_name, y_train[i], round(y_pred[i], 2), round(y_pred[i] - y_train[i], 2)))

    data = sorted(data, key=lambda x: (x[5]))
    print()
    print(f"ID\t{pad_display('曲名', 32)}\t難度\t等級\t預測\t誤差")
    for idx, s, d, t, p, diff in data:
        print(f"{idx}\t{pad_display(s[:30], 35)}\t{d}\t{t}\t{p}\t{diff}")

    # y_pred = np.round(y_pred, 2)
    # print("\t".join(map(str, np.reshape(y_pred, (-1,)))))
    # print("\t".join(map(str, np.reshape(y_test, (-1,)))))

    print()
    print("TRAIN Mean Squared Error:", mse)
    print("TRAIN R² Score:", r2)


Train size:578
Test size:65

ID	曲名                            	難度	等級	預測	誤差
141	Break Through the Barrier          	chaos	13.6	12.170000076293945	-1.43
376	DON'T STOP ROCKIN'                 	chaos	12.8	11.710000038146973	-1.09
589	すゝめ☆クノイチの巻                	chaos	14.6	13.699999809265137	-0.9
138	dimensionalize nervous breakdo     	chaos	13.8	12.920000076293945	-0.88
324	dynamo                             	chaos	15.0	14.130000114440918	-0.87
252	II                                 	chaos	15.0	14.140000343322754	-0.86
137	Jazzy Glitch Machine               	chaos	13.8	12.979999542236328	-0.82
82	Online                             	chaos	13.4	12.59000015258789	-0.81
576	Doldrums                           	chaos	14.2	13.430000305175781	-0.77
297	DON'T LISTEN TO THIS WHILE DRI     	chaos	13.6	12.979999542236328	-0.62
409	#Interstellar_Believer             	chaos	15.0	14.380000114440918	-0.62
95	Liberation <GLITCH>                	glitch	15.8	15.229999542236328	-0.57
450	ANiMA                

In [86]:
# 儲存模型
import pickle
from sklearn.ensemble import RandomForestRegressor

with open('CytusIIDifficultyAnalyzer.pkl', 'wb') as f:
    pickle.dump(model, f)


In [58]:
import zipfile
import json

def getCytoidLevelChart(levelID):
    print("Loading Cytoidlevel:", levelID)
    if not levelID.endswith(".cytoidlevel"):
        levelID += ".cytoidlevel"
    # 指定 zip 檔路徑
    zip_path = levelID

    # 開啟 zip 檔
    with zipfile.ZipFile(zip_path, 'r') as z:
        # 列出所有檔案
        # print("檔案清單:", z.namelist())
        
        # 讀取其中一個檔案內容（假設是文字檔）
        
        with z.open("level.json") as f:
            data = json.loads(f.read().decode("utf-8"))
            if len(data["charts"]) == 1:
                chart_diff_id = 0
            else:
                chart_diff_id = max(
                    range(len(data["charts"])), 
                    key=lambda i: data["charts"][i]["difficulty"]
                )
                print("Multi level")
                print("Using Diff:", data["charts"][chart_diff_id]["difficulty"])
            # chart_diff_id = 0
            chart_meta = data["charts"][chart_diff_id]
            # print(chart_meta)
            if "storyboard" in chart_meta.keys():
                with z.open(chart_meta["storyboard"]["path"]) as sb:
                    data = json.loads(sb.read().decode("utf-8"))
                    if "chartBackup" in data.keys():
                        print("Has Backup")
                        return data["chartBackup"]

            with z.open(chart_meta["path"]) as cht:
                chart = json.loads(cht.read().decode("utf-8"))
                return chart

# getCytoidLevelChart("sn.memoria")
pass

In [66]:
import pickle
with open('CytusIIDifficultyAnalyzer.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

CHART_FILE_NAME = "he.erabreath"
IS_CYTOID = True
# IS_CYTOID = False

if IS_CYTOID:
    GLITCH = getCytoidLevelChart(CHART_FILE_NAME)
else:
    with open(CHART_FILE_NAME) as f:
        GLITCH = json.load(f)
X_target, _ = get_all_feature([GLITCH])
y_target = loaded_model.predict(X_target)
print()
print(y_target)

Loading Cytoidlevel: whigrac.senju


Processing charts: 100%|██████████| 1/1 [00:00<00:00, 16.72it/s]


[15.756642]


In [ ]:
# 特徵的重要性
import numpy as np
import matplotlib.pyplot as plt

# 假設你已經有：
# X: 原始特徵 (NumPy array)
# model.feature_importances_: 各特徵的重要性
# feature_name: 對應的特徵名稱（list 或 np.array）

# --- Step 1: 取得前 20 特徵畫圖用 ---
importance_array = model.feature_importances_
feature_name = np.array(feature_name)

# 找出前 20 名重要特徵的索引（由大到小）
top_20_indices = np.argsort(importance_array)[::-1][:]

# 畫圖
plt.figure(figsize=(10, 6))
plt.barh(feature_name[top_20_indices][::-1], importance_array[top_20_indices][::-1])
plt.xlabel("Feature Importance")
plt.title("Top 20 Important Features")
plt.tight_layout()
plt.show()

# --- Step 2: 建立 X_top_10_feature ---
top_10_indices = top_20_indices[:10]  # 取出前10的 index
X_top_10_feature = X[:, top_10_indices]
top_10_feature_names = feature_name[top_10_indices]

# 可選：列出名稱與 shape 確認
print("前 10 特徵名稱：", top_10_feature_names)
print("X_top_10_feature.shape =", X_top_10_feature.shape)


In [22]:
# 找特徵相似的歌，沒什麼用
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances


# 1. 標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) * importance_array


song_id = 632

# 計算所有歌曲之間的歐氏距離（越小越相似）
dists = euclidean_distances(X_scaled, X_scaled)

# 對指定歌曲，找距離最小的其他歌曲（自己距離為 0，一定最小）
sort_idx = np.argsort(dists[song_id])  # 從小到大排序

# 列出前 8 首最相似（跳過自己）
for j in range(1, 9):
    print("最相似的歌曲名：", all_song_name[sort_idx[j]])


最相似的歌曲名： Is This Real Life?
最相似的歌曲名： Oshama Scramble!
最相似的歌曲名： DJ Mashiro is dead or alive
最相似的歌曲名： La Prière
最相似的歌曲名： For You the Bellz Toll
最相似的歌曲名： Re:boot
最相似的歌曲名： Heart Factorisation
最相似的歌曲名： felzione


In [ ]:
# 觀察特徵分布
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 假設 X 是 numpy array，Y 是 numpy array（目標值）
df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

n_features = df.shape[1]
n_cols = 4
n_rows = int(np.ceil((n_features + 1) / n_cols))  # +1 是為了放 Y

plt.figure(figsize=(n_cols * 4, n_rows * 3))

# 繪製 X 各特徵分布
for i, col in enumerate(df.columns):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df[col], bins=30, kde=True, color="steelblue")
    plt.title(feature_name[i])
    plt.tight_layout()

# 繪製 Y 分布
plt.subplot(n_rows, n_cols, n_features + 1)
sns.histplot(Y, bins=30, kde=True, color="orange")
plt.title("Target Y Distribution")
plt.tight_layout()

plt.show()


In [ ]:
# 特徵跟難度相關性
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 將 X 轉為 DataFrame（列為樣本，欄為特徵）
df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

n_features = df.shape[1]
n_cols = 3
n_rows = int(np.ceil(n_features / n_cols))

highlight_ids = [632, 409, 177, 117, 208, 213]  # 多筆樣本 index
colors = plt.get_cmap("gist_rainbow", len(highlight_ids))

plt.figure(figsize=(n_cols * 4, n_rows * 3))

for h in highlight_ids:
  print(f"{h}\t{all_song_name[h]}")

for i, col in enumerate(df.columns):
    plt.subplot(n_rows, n_cols, i + 1)
    # 全部點
    sns.scatterplot(x=df[col], y=Y, color='steelblue', edgecolor='k', s=5, alpha=0.25)
    # 高亮每個指定的樣本
    for j, idx in enumerate(highlight_ids):
        x_val = df[col].iloc[idx]
        y_val = Y[idx]
        plt.scatter(df[col].iloc[idx], Y[idx],
                    color=colors(j+1), edgecolors='black',
                    s=40, marker='o', label=f"ID {idx}", zorder=10)
        plt.text(x_val, y_val, f"{idx}", fontsize=9, color='black',
                 weight='bold', ha='left', va='bottom')
    plt.title(f"{i}. {feature_name[i]}")
    plt.xlabel("")
    plt.ylabel("Y")

plt.tight_layout()
plt.show()

In [ ]:
# 列出特徵排序
feature_id = 21
arg_sort = np.argsort(X.T[feature_id])
print(feature_name[feature_id])
print()
for i in reversed(arg_sort[:]):
    print(f"{i}\t{pad_display(all_song_name[i][:25], 32)}\t{np.round(X.T[feature_id, i], 3)}\t{Y[i]}")

In [ ]:
# 檢視歌曲特徵
n_samples, n_features = X.shape
rank_X = np.zeros_like(X, dtype=int)

for j in range(n_features):
    # 取得該特徵的排序（從小到大）
    sorted_idx = np.argsort(X[:, j])
    # 分配名次：名次從 0 開始 → 最小值是 0，最大值是 n-1
    for rank, i in enumerate(sorted_idx):
        rank_X[i, j] = rank
song_id = 495
print(all_song_name[song_id])
for i, f_n in enumerate(feature_name):
    print(f"{f_n:<30}\t{len(all_song_name) - rank_X[song_id, i]}")

In [81]:
# KFold 測試
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 設定折數
kf = KFold(n_splits=10, shuffle=True)

candidate = [
    'burst_p90',
    'page_space_p90_score',
    'page_space_third_score',
    'burst_song_avg',
    'complex_beat_count',
    'Drag-child',
    'burst_LR_low_max',
    'double_count',
    'SONG_LENGTH',
    'burst_endurance_8',
    'CDrag-head',
    'MAIN_BPM',
    'burst_fifth'
]
feat2idx = {f: i for i, f in enumerate(feature_name)}
idxs = [feat2idx[feat] for feat in candidate]
X_subset = X[:, idxs]

r2_scores = []
fold = 1

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    # model = TabNetRegressor(optimizer_params=dict(lr=3e-2), verbose=0)
    # y_train = np.reshape(y_train, (-1, 1))
    # y_test = np.reshape(y_test, (-1, 1))
    # model.fit(
    #     X_train, y_train,
    #     eval_set=[(X_test, y_test)],
    #     eval_metric=['mse'],
    #     max_epochs=3000,
    #     patience=100,
    #     batch_size=1024,
    #     virtual_batch_size=128,
    #     num_workers=0,
    #     augmentations=augmentations,
    #     drop_last=False
    # )

    # model = getVoteModel()
    # model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_test = y_test.reshape(-1)
    y_pred = y_pred.reshape(-1)

    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)


# 顯示總結
print("\n==============================================")
print("平均 R² 分數：", round(np.mean(r2_scores), 4))


平均 R² 分數： 0.8312


In [ ]:
# 特徵選擇
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

def forward_feature_selection_repeat_kfold(
    X, Y, feature_names,
    n_splits=5,
    repeats=3,
    max_features=None,
    tol=1e-3
):
    feat2idx = {f: i for i, f in enumerate(feature_names)}
    selected = []
    remaining = set(feature_names)
    best_r2 = -np.inf

    while remaining:
        best_feat = None
        best_feat_r2 = best_r2

        for f in sorted(remaining):  # 排序保證確定性
            candidate = selected + [f]
            idxs = [feat2idx[feat] for feat in candidate]
            X_subset = X[:, idxs]

            r2_repeats = []

            for repeat in range(repeats):
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=42 + repeat)
                r2_scores = []

                for train_idx, test_idx in kf.split(X_subset):
                    X_train, X_test = X_subset[train_idx], X_subset[test_idx]
                    y_train, y_test = Y[train_idx], Y[test_idx]

                    # model = RandomForestRegressor(n_estimators=100, random_state=42)
                    model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
                    model.fit(X_train, y_train)

                    # model = TabNetRegressor(optimizer_params=dict(lr=3e-2), verbose=0)
                    # y_train = np.reshape(y_train, (-1, 1))
                    # y_test = np.reshape(y_test, (-1, 1))
                    # model.fit(
                    #     X_train, y_train,
                    #     eval_set=[(X_test, y_test)],
                    #     eval_metric=['mse'],
                    #     max_epochs=3000,
                    #     patience=100,
                    #     batch_size=1024,
                    #     virtual_batch_size=128,
                    #     num_workers=0,
                    #     augmentations=augmentations,
                    #     drop_last=False
                    # )

                    y_pred = model.predict(X_test)
                    r2_scores.append(r2_score(y_test, y_pred))

                r2_repeats.append(np.mean(r2_scores))

            avg_r2 = np.mean(r2_repeats)

            if avg_r2 > best_feat_r2 + tol:
                best_feat_r2 = avg_r2
                best_feat = f

        if best_feat is not None:
            selected.append(best_feat)
            remaining.remove(best_feat)
            best_r2 = best_feat_r2
            print(f"加入特徵 {best_feat}，平均 R² 提升到 {best_r2:.4f}")
            if max_features and len(selected) >= max_features:
                break
        else:
            break

    return selected, best_r2

selected_features, best_r2 = forward_feature_selection_repeat_kfold(
    X, Y, feature_name,
    n_splits=10,
    repeats=5,         # 每個特徵做5次 KFold，平均 R²
    max_features=20,   # 最多挑 10 個特徵
    tol=1e-3           # 最小提升幅度
)

print("選擇的特徵：", selected_features)
print("最佳平均 R²：", best_r2)

warnings.resetwarnings()

## RandomForestRegressor

| 加入順序 | 特徵名稱                     | 平均 R²  |
| ---- | ------------------------ | ------ |
| 1    | `burst_p90`              | 0.5520 |
| 2    | `page_space_p90_score`   | 0.7033 |
| 3    | `burst_fifth`            | 0.7471 |
| 4    | `SONG_LENGTH`            | 0.7692 |
| 5    | `complex_beat_count`     | 0.7814 |
| 6    | `burst_song_avg`         | 0.7961 |
| 7    | `page_space_third_score` | 0.8088 |
| 8    | `burst_LR_low_max`       | 0.8125 |
| 9    | `page_tick_changes`      | 0.8142 |
| 10   | `MAIN_BPM`               | 0.8157 |

**最佳平均 R²**：`0.8156595084003785`



| 加入順序 | 特徵名稱                      | 平均 R² 提升後數值 |
| ---- | ------------------------- | ----------- |
| 1    | burst\_p90                | 0.5571      |
| 2    | page\_space\_p90\_score   | 0.6944      |
| 3    | burst\_fifth              | 0.7434      |
| 4    | page\_space\_third\_score | 0.7694      |
| 5    | complex\_beat\_count      | 0.7891      |
| 6    | burst\_endurance\_16      | 0.8007      |
| 7    | CDrag-head                | 0.8087      |
| 8    | Drag-child                | 0.8148      |
| 9    | burst\_song\_avg          | 0.8251      |
| 10   | Drag-head                 | 0.8275      |
| 11   | CDrag-child               | 0.8295      |

**最佳平均 R²：** `0.8295`



| 加入順序 | 特徵名稱                      | 平均 R² 提升後數值 |
| ---- | ------------------------- | ----------- |
| 1    | burst\_p90                | 0.5571      |
| 2    | page\_space\_p90\_score   | 0.6944      |
| 3    | burst\_fifth              | 0.7434      |
| 4    | burst\_endurance\_8       | 0.7699      |
| 5    | complex\_beat\_count      | 0.7918      |
| 6    | page\_space\_third\_score | 0.8045      |
| 7    | CDrag-child               | 0.8118      |
| 8    | Drag-child                | 0.8187      |
| 9    | burst\_song\_avg          | 0.8253      |
| 10   | page\_endurance\_24       | 0.8288      |
| 11   | SONG\_LENGTH              | 0.8312      |
| 12   | all\_rhythm\_entropy      | 0.8322      |

**最佳平均 R²：** `0.8322`

## XGBRegressor

| 加入順序 | 特徵名稱                     | 加入後平均 R² |
| ---- | ------------------------ | -------- |
| 1    | burst_p90              | 0.5997   |
| 2    | page_space_p90_score   | 0.6951   |
| 3    | page_space_third_score | 0.7278   |
| 4    | burst_song_avg         | 0.7570   |
| 5    | complex_beat_count     | 0.7808   |
| 6    | Drag-child             | 0.8054   |
| 7    | burst_LR_low_max       | 0.8167   |
| 8    | double_count           | 0.8225   |
| 9    | SONG_LENGTH            | 0.8284   |
| 10   | burst_endurance_8      | 0.8362   |
| 11   | CDrag-head             | 0.8383   |
| 12   | MAIN_BPM               | 0.8397   |
| 13   | burst_fifth            | 0.8434   |

**最佳平均 R²：** `0.8434`

